In [42]:
import pandas as pd
import regex as re
import string

In [16]:
# Question ID and Answer text pair
df = pd.read_csv("train/FiQA_train_doc_final.tsv", sep="\t")

df.head(5)

,Unnamed: 0,docid,doc,timestamp
0,0,3,I'm not saying I don't like the idea of on-the...,Oct 03 '12 at 14:56
1,1,31,So nothing preventing false ratings besides ad...,Sep 01 '17 at 13:36
2,2,56,You can never use a health FSA for individual ...,Jun 9 '14 at 17:37
3,3,59,Samsung created the LCD and other flat screen ...,Dec 27 at 01:37
4,4,63,Here are the SEC requirements: The federal sec...,Jul 14 '14 at 8:10


In [138]:
collection = df[['docid', 'doc']]
collection = collection.sort_values(by=['docid'])

def process(doc):
    
    doc = str(doc)
    
    x = re.sub('[!&\'"@#()\*\+,/:;\-<=>?@[\]\^_`{\}~]', '', doc)
    
    return x

def remove_period(doc):
    
    doc = str(doc)
    
    pattern = re.compile(r'\.(\s[A-Z])')
    
    x = pattern.sub(r'\1', doc)
    
    if x[-1] == ".":
        x = x[:-1]
        
    return x

collection['doc'] = collection['doc'].apply(process)

collection['doc'] = collection['doc'].apply(remove_period)

collection['doc'] = collection['doc'].str.lower()


a = collection['doc'][0]

print(a)

collection.head(10)

im not saying i dont like the idea of onthejob training too but you cant expect the company to do that training workers is not their job  theyre building software perhaps educational systems in the u.s. or their students should worry a little about getting marketable skills in exchange for their massive investment in education rather than getting out with thousands in student debt and then complaining that they arent qualified to do anything


,docid,doc
0,3,im not saying i dont like the idea of onthejob...
1,31,so nothing preventing false ratings besides ad...
2,56,you can never use a health fsa for individual ...
3,59,samsung created the lcd and other flat screen ...
4,63,here are the sec requirements the federal secu...
5,100,only relevant to those with fantasy economy te...
6,108,futures contracts are a member of a larger cla...
7,125,this month when you join scentsy you get a fre...
8,132,whenever you pay or withdraw some fund from yo...
9,138,so you asked him in 2010 how he was gong to co...


In [30]:
with open("fiqa-passage/collection.tsv",'w') as write_tsv:
    write_tsv.write(collection.to_csv(sep='\t', index=False, header=False))

In [3]:
# Question ID and Answer ID pair
df = pd.read_csv("train/FiQA_train_question_doc_final.tsv", sep="\t")

df.head(5)

,Unnamed: 0,qid,docid
0,0,0,18850
1,1,1,14255
2,2,2,308938
3,3,3,296717
4,4,3,100764


In [4]:
# Question ID and Question text pair
df = pd.read_csv("train/FiQA_train_question_final.tsv", sep="\t")

df.head(5)

,Unnamed: 0,qid,question,timestamp
0,0,0,What is considered a business expense on a bus...,Nov 8 '11 at 15:14
1,1,1,Claiming business expenses for a business with...,May 13 '14 at 13:17
2,2,2,Transferring money from One business checking ...,Jan 20 '16 at 20:31
3,3,3,Having a separate bank account for business/in...,Mar 1 at 0:24
4,4,4,Business Expense - Car Insurance Deductible Fo...,Mar 4 at 0:26
